In [ ]:
!git clone https://github.com/oKatanaaa/GCT

In [ ]:
!pip install --upgrade git+https://github.com/oKatanaaa/MakiFlow.git

In [ ]:
%tensorflow_version 1.x

In [ ]:
#from GCT.tools import load_cifar10
from makiflow.layers import *
from makiflow.models import Classificator
from makiflow.generators.classification im
from makiflow.base import MakiLayer
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
(Xtrain, Ytrain), (Xtest, Ytest), (X_mean, X_std) = load_cifar10()

In [ ]:
BATCH_SIZE = 64

In [ ]:
def block(x, id, in_f, out_f):
    x_skip = x
    x_main = x
    x_main = ConvLayer(kw=3, kh=3, in_f=in_f, out_f=out_f, name=f'conv_main{id}_1')(x_main)
    x_main = BatchNormLayer(D=out_f, name=f'bn_main{id}_1')(x_main)
    x_main = ConvLayer(kw=3, kh=3, in_f=out_f, out_f=out_f, name=f'conv_main{id}_2')(x_main)
    x_main = BatchNormLayer(D=out_f, name=f'bn_main{id}_2')(x_main)
    x_main = MaxPoolLayer(name=f'mp{id}')(x_main)

    x_skip = ConvLayer(kw=1, kh=1, in_f=in_f, out_f=out_f, stride=2, activation=None, name=f'conv_skip{id}')(x_skip)
    x_skip = BatchNormLayer(D=out_f, name=f'bn_skip{id}_1')(x_skip)

    x_sum = SumLayer(name=f'sum{id}')([x_main, x_skip])
    return x_sum

In [ ]:
inp1 = InputLayer(input_shape=[BATCH_SIZE, 32, 32, 3], name='input')
x1 = block(inp1, id=1, in_f=3, out_f=32)
# 16x16
x1 = block(x1, id=2, in_f=32, out_f=64)
# 8x8
x1 = block(x1, id=3, in_f=64, out_f=128)
# 4x4
x1 = block(x1, id=4, in_f=128, out_f=256)
# 2x2
x1 = FlattenLayer('flat')(x1)
# 4*256 = 1024
x1 = DenseLayer(in_d=1024, out_d=256, name='projection_head')(x1)
x1 = DenseLayer(in_d=256, out_d=10, activation=None, name='classification_head')(x1)

In [ ]:
model1 = Classificator(input=inp1, output=x1)
sess = tf.Session()
model.set_session(sess)

In [ ]:
opt = tf.train.AdamOptimizer()

In [ ]:
gen = cycle_generator(Xtrain, Ytrain.reshape(-1), BATCH_SIZE)

In [ ]:
## Train a model with classic architecture for the comparison later

In [ ]:
info1 = model1.genfit_ce(gen, (Xtest, Ytest.reshape(-1)), optimizer=opt, epochs=10, iter=400)

In [ ]:
class NewLayer(MakiLayer):    
    def _forward(self, x, computation_mode=MakiLayer.INFERENCE_MODE):
        pass
    
    def _training_forward(self, x):
        pass
    
    @staticmethod
    def build(params: dict):
        pass

    def to_dict(self):
        pass

In [ ]:
def new_block(x, id, in_f, out_f):
    # ADD NEW LAYER HERE
    x_skip = x
    x_main = x
    x_main = ConvLayer(kw=3, kh=3, in_f=in_f, out_f=out_f, name=f'conv_main{id}_1')(x_main)
    x_main = BatchNormLayer(D=out_f, name=f'bn_main{id}_1')(x_main)
    x_main = ConvLayer(kw=3, kh=3, in_f=out_f, out_f=out_f, name=f'conv_main{id}_2')(x_main)
    x_main = BatchNormLayer(D=out_f, name=f'bn_main{id}_2')(x_main)
    x_main = MaxPoolLayer(name=f'mp{id}')(x_main)

    x_skip = ConvLayer(kw=1, kh=1, in_f=in_f, out_f=out_f, stride=2, activation=None, name=f'conv_skip{id}')(x_skip)
    x_skip = BatchNormLayer(D=out_f, name=f'bn_skip{id}_1')(x_skip)

    x_sum = SumLayer(name=f'sum{id}')([x_main, x_skip])
    return x_sum

In [ ]:
inp2 = InputLayer(input_shape=[BATCH_SIZE, 32, 32, 3], name='input')
x2 = block(inp2, id=1, in_f=3, out_f=32)
# 16x16
x2 = block(x2, id=2, in_f=32, out_f=64)
# 8x8
x2 = block(x2, id=3, in_f=64, out_f=128)
# 4x4
x2 = block(x2, id=4, in_f=128, out_f=256)
# 2x2
x2 = FlattenLayer('flat')(x2)
# 4*256 = 1024
x2 = DenseLayer(in_d=1024, out_d=256, name='projection_head')(x2)
x2 = DenseLayer(in_d=256, out_d=10, activation=None, name='classification_head')(x2)

In [ ]:
model2 = Classificator(input=inp2, output=x2)
model2.set_session(sess)

In [ ]:
info2 = model1.genfit_ce(gen, (Xtest, Ytest.reshape(-1)), optimizer=opt, epochs=10, iter=400)

In [ ]:
plt.plot(info1['test errors'], label='model1 errors')
plt.legend()
plt.plot(info2['test errors'], label='model2 errors')
plt.legend()